# Machine Learning

## Regularización
Tarea 5 : Gustavo Augusto Mondragón Sánchez : 136894

### Introducción:
#### Regularización
En modelos estadísticos, la regularización se refiere al proceso de introducir información adicional para solucionar un problema mal definido o para impedir el sobreentrenamiento. Es decir, ayuda a evitar que nuestro modelo se aprenda de memoria los datos.

Pero ¿cómo surge un problema de sobreentrenamiento? intentemos con un ejemplo:

Cuando uno trata de modelar un problema como, por ejemplo, predecir el número de glóbulos rojos en la sangre, lo primero que intentará será usar un modelo de regresión lineal, con el peso como variable independiente y el número de glóbulos rojos como la dependiente. Lo más probable es que el modelo falle ya que es demasiado simple.
Puedes oensar entonces, "tengo otras variables que pueden ayudar como la edad, enfermedades, tipo de sangre, estatura".
Con esta adición de variables e información el modelo se volverá más complejo. En el modelo mides la exactitud con respecto a una métrica $L(X,y)$ donde $X$ es la matriz por tupla de información y $Y$ es el vector de observaciones (en nuestro caso, el número de glóbulos rojos).
Parece que el modelo es bueno... pero no tanto... Así que decides agregar más información como domicilio, nacionalidad, escolaridad, comida favorita, hora a la que despierta, nombre de sus padres, etc.
El modelo ahora se comporta extraordinariamente bien... pero probablemente está sobreentrenado. Lo más seguro es que tenga poco poder de predicción para información nueva y poca capacidad de generalización. Se aprendió toda la información, incluso el ruido de fondo.

Una de las formas de atacar este problema es con las técnicas de regularización.

Puedes penalizar la métrica de ajuste agregando un múltiplo de $L_1(LASSO)$ O una norma de $L_2(Ridge)$ de los pesos del vector w (parámetros de la regresión). Se consigue la siguiente ecuación:
$$L(X,y) + λN(w) ~~~~~ (N es L1, L2 o cualquier otra norma indistintamente)$$ 

Esto te permitirá evitar el sobreenetrenamiento. 
Para la siguiente etapa de encontrar el $λ$ adecuado se puede hacer cross-validation al variar $λ$ con tu set de validación y buscar el que minimice la métrica asignada.





[Wikipedia, Ridge](https://en.wikipedia.org/wiki/Tikhonov_regularization)
[Wikipedia, Regularización](https://en.wikipedia.org/wiki/Regularization_(mathematics))
[Quora](https://www.quora.com/What-is-regularization-in-machine-learning)

### Objetivos:
* Implementar el algoritmo iterativo incremental regularizado para calcular una regresión lineal.
* Escalar los datos usando StandardScaler
* Comparar el error y los pesos resultantes para una lambda=0 y una lambda=0.001
* Programar el minibatch para regresión lineal iterativa.

In [2]:
#Librerías necesarias:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets, linear_model, preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

### Estructura de los datos: 

* Una variable dependiente continua Y
* 36 variables independientes continuas $X_i, ~~~ i=1,...,36$

In [168]:
#Cargar los datos desde repositorio online:
dataNl = pd.read_csv('https://raw.githubusercontent.com/ClaseML-2017/MaterialyTareas/master/datos/regLinPoli2.csv',delimiter=',') # Leer el archivo
#Descripción previa de los datos:
print dataNl.describe()

                X1             X2           X3           X4           X5  \
count  1029.000000    1029.000000  1029.000000  1029.000000  1029.000000   
mean    -12.492680  327718.372035     2.565422    -0.018122     1.235180   
std     572.608894  298395.344783     0.424105     0.715733    58.709228   
min    -992.795572       1.098476     0.020395    -0.999991  -100.000000   
25%    -499.000060   59791.243170     2.388319    -0.760277   -51.000000   
50%     -27.174576  241226.533900     2.691213    -0.029038     3.000000   
75%     477.216100  547320.664300     2.869121     0.714801    52.000000   
max     998.297367  996597.632200     2.999260     0.999992   100.000000   

                X6           X7           X8           X9          X10  \
count  1029.000000   1029.00000  1029.000000  1029.000000  1029.000000   
mean   -153.782162 -23648.95337    -2.565422     0.018122    -4.108330   
std       0.000000      0.00000     0.424105     0.715733   584.648570   
min    -153.782162 

### División del conjunto de datos en entrenamiento y validación.

In [169]:
np.random.seed(12345)     #Para revisiones de los resultados futuras

#División de los datos 
trainx, testx, trainy, testy = train_test_split(dataNl[dataNl.columns[0:-1]],dataNl[dataNl.columns[-1]], train_size=0.75)

#Datos estandarizados
scaler = preprocessing.StandardScaler().fit(trainx)
trainxE=scaler.transform(trainx)
testxE=scaler.transform(testx)

### Creación del modelo LMS REGULARIZADO:

In [170]:
def LMS(x,y,eta,lam):
    x = np.insert(np.array(x), 0, 1, axis=1)
    y = np.array(y)
    nr = x.shape[0]
    nc = x.shape[1]
    w = np.random.random((1,nc))
    for i in range(0,nr):
        e = (y[i] - np.dot(x[i],w.T))
        w = w + eta*e*x[i]-lam*w
    print 'Coeficientes: \n',w.T
    return w


def testLMS(x,y,w,indiceG=1):
    x = np.insert(np.array(x), 0, 1, axis=1)
    nr = x.shape[0]
    y = np.array(y)
    pred = np.dot(x,w.T)
    plt.scatter(x[:,indiceG],y, color='black')
    plt.scatter(x[:,indiceG],pred)
    ecm = (1.0/len(y))*np.abs(np.sum(np.array(y)-pred))
    print '\n Error medio: %f' % ecm
    plt.title('Regresion lineal usando parametros de W')
    print '\n Ordenada: %f, Pendiente: %f' % (w[0,0], w[0,1])
    if indiceG>0: 
        plt.show()
    return ecm


### Validación del modelo eta = 0.01, lambda = 0

In [171]:
w = LMS(trainxE,trainy,.01,0)
testLMS(testxE,testy,w,-1)

Coeficientes: 
[[  2.53863217e+18]
 [  1.89286605e+20]
 [ -1.35096802e+19]
 [  9.42058099e+18]
 [  7.02147975e+18]
 [  2.26497643e+18]
 [  2.53863217e+18]
 [ -2.53863217e+18]
 [ -9.42058099e+18]
 [ -7.02147975e+18]
 [  1.51830503e+18]
 [  8.40560069e+18]
 [  2.18317860e+19]
 [ -2.07502344e+19]
 [  8.99352047e+18]
 [ -3.00177574e+19]
 [ -1.67151787e+19]
 [  8.20619296e+18]
 [  2.14701700e+18]
 [ -7.61341308e+18]
 [  1.10261376e+19]
 [  1.36835838e+18]
 [ -1.77949682e+19]
 [  2.70505983e+18]
 [ -1.52855373e+19]
 [ -1.00464657e+19]
 [ -1.54399625e+19]
 [  1.31947921e+19]
 [  3.93849292e+18]
 [ -9.02649139e+18]
 [ -1.44332205e+18]
 [  2.78706894e+18]
 [  7.48530545e+18]
 [ -9.23705474e+18]
 [  3.67903232e+18]
 [ -2.47477950e+19]
 [  1.92957927e+19]
 [ -2.62234016e+18]
 [  4.05129915e+18]]

 Error medio: 3971414912492378259456.000000

 Ordenada: 2538632171882640384.000000, Pendiente: 189286605411517497344.000000


3.9714149124923783e+21

### Validación del modelo eta = 0.01, lambda = 0.01

In [172]:
w = LMS(trainxE,trainy,.01,0.001)
testLMS(testxE,testy,w,-1)

Coeficientes: 
[[  2.43222540e+18]
 [  1.71920693e+20]
 [ -4.97954530e+18]
 [  7.76614506e+18]
 [  7.14202046e+18]
 [  1.97491107e+18]
 [  2.43222540e+18]
 [ -2.43222540e+18]
 [ -7.76614506e+18]
 [ -7.14202046e+18]
 [  8.31859137e+17]
 [  9.38300988e+18]
 [  2.06900111e+19]
 [ -1.89122913e+19]
 [  6.19802953e+18]
 [ -2.81932816e+19]
 [ -1.71358621e+19]
 [  5.04474320e+18]
 [ -5.87905725e+16]
 [ -5.71482400e+18]
 [  1.11384924e+19]
 [  1.79410430e+17]
 [ -1.67537990e+19]
 [  2.54035138e+18]
 [ -1.36336849e+19]
 [ -7.72976828e+18]
 [ -1.51201100e+19]
 [  1.20363185e+19]
 [  4.69929444e+18]
 [ -8.19193847e+18]
 [ -2.90666612e+18]
 [  5.42248694e+17]
 [  9.36368288e+18]
 [ -7.72248900e+18]
 [  3.75956658e+18]
 [ -2.36294293e+19]
 [  1.92586302e+19]
 [ -2.41274619e+18]
 [  2.87799926e+18]]

 Error medio: 3417814908782582956032.000000

 Ordenada: 2432225404555486720.000000, Pendiente: 171920692668388048896.000000


3.417814908782583e+21

## Con lambda más grande los coeficientes de algunas variables independientes se hace cada vez más cercano a cero.